複数のコンポジションを色々組み合わせてみる

In [1]:
from pxr import Usd,Sdf

In [2]:
stage = Usd.Stage.CreateInMemory()
stage.Reload()
layer = stage.GetRootLayer()

In [3]:
# Local(そのレイヤーに定義を作る)
prim = stage.DefinePrim("/DefPrim")
stage.SetDefaultPrim(prim)

In [4]:
# Variantのパターンを追加
vset = prim.GetVariantSets().AddVariantSet('hogehoge')
vset.AddVariant('cube')
vset.AddVariant('sphere')

True

In [5]:
# Inherits用のPrimを作る
cls = stage.CreateClassPrim("/basePrim")
cls.CreateAttribute('addClassParam',Sdf.ValueTypeNames.String).Set("hogehoge")

refBasePrim = stage.CreateClassPrim("/refBasePrim")
refBasePrim.CreateAttribute("addClassParam",Sdf.ValueTypeNames.String).Set("refValue")

True

In [6]:
# Variantを作る
# Variant以下にReferenceがある場合、Vのほうが強いので
# Variantで選んだ選択肢以下のリファレンスがコンポジションされる
vset.SetVariantSelection('cube')
# Variant以下にReference
with vset.GetVariantEditContext():
    cprim = stage.DefinePrim(prim.GetPath().AppendChild("cube"))
    cprim.GetReferences().AddReference('cube.usda')
    cprim.GetInherits().AddInherit('/basePrim')

vset.SetVariantSelection('sphere')
# Variant以下にReference
with vset.GetVariantEditContext():
    cprim = stage.DefinePrim(prim.GetPath().AppendChild("sphere"))
    cprim.GetReferences().AddReference('sphere.usda')
    cprim.GetInherits().AddInherit('/basePrim')

print(prim)
# Inherits付きのPrimを作る
cprim = stage.DefinePrim(prim.GetPath().AppendChild("childA"))
cprim.GetInherits().AddInherit('/basePrim') 

cprim = stage.DefinePrim(prim.GetPath().AppendChild("childB"))
cprim.GetInherits().AddInherit('/basePrim') 

# ReferenceとInheritsを同時に使う
cprim = stage.DefinePrim(prim.GetPath().AppendChild("childC"))
cprim.GetInherits().AddInherit('/basePrim') 
cprim.GetReferences().AddInternalReference("/refBasePrim")

Usd.Prim(</DefPrim>)


True

In [7]:
print(layer.ExportToString())
layer.Export("usd/baseLayer.usda")

#usda 1.0
(
    defaultPrim = "DefPrim"
)

def "DefPrim" (
    variants = {
        string hogehoge = "sphere"
    }
    prepend variantSets = "hogehoge"
)
{
    def "childA" (
        prepend inherits = </basePrim>
    )
    {
    }

    def "childB" (
        prepend inherits = </basePrim>
    )
    {
    }

    def "childC" (
        prepend inherits = </basePrim>
        prepend references = </refBasePrim>
    )
    {
    }
    variantSet "hogehoge" = {
        "cube" {
            def "cube" (
                prepend inherits = </basePrim>
                prepend references = @cube.usda@
            )
            {
            }

        }
        "sphere" {
            def "sphere" (
                prepend inherits = </basePrim>
                prepend references = @sphere.usda@
            )
            {
            }

        }
    }
}

class "basePrim"
{
    custom string addClassParam = "hogehoge"
}

class "refBasePrim"
{
    custom string addClassParam = "refValue"
}




True

In [8]:
# サブレイヤー
subStage = Usd.Stage.CreateInMemory()
subLayer = subStage.GetRootLayer()
subLayer.subLayerPaths = ['usd/baseLayer.usda']
subLayer.Export("usd/root.usda")

# サブレイヤーしたレイヤーでClassに対してCreateAttribute
subPrim = subStage.GetPrimAtPath("/basePrim")
subPrim.CreateAttribute('addAttr',Sdf.ValueTypeNames.Int).Set(100)

# SphereのときはOverで値を編集する
over = subStage.OverridePrim("/DefPrim/sphere")
over.GetAttribute('addClassParam').Set('fugafuga')

childB = subStage.GetPrimAtPath("/DefPrim/childB")
childB.GetAttribute('addClassParam').Set('setSubLayer')

True

In [9]:
print(subStage.GetRootLayer().ExportToString())

#usda 1.0
(
    subLayers = [
        @usd/baseLayer.usda@
    ]
)

over "basePrim"
{
    custom int addAttr = 100
}

over "DefPrim"
{
    over "sphere"
    {
        custom string addClassParam = "fugafuga"
    }

    over "childB"
    {
        custom string addClassParam = "setSubLayer"
    }
}




In [10]:
print(subStage.Flatten().ExportToString())

#usda 1.0
(
    doc = """Generated from Composed Stage of root layer 
"""
)

def "DefPrim"
{
    def Sphere "sphere"
    {
        custom int addAttr = 100
        custom string addClassParam = "fugafuga"
    }

    def "childA"
    {
        custom int addAttr = 100
        custom string addClassParam = "hogehoge"
    }

    def "childB"
    {
        custom int addAttr = 100
        custom string addClassParam = "setSubLayer"
    }

    def "childC"
    {
        custom int addAttr = 100
        custom string addClassParam = "hogehoge"
    }
}

class "basePrim"
{
    custom int addAttr = 100
    custom string addClassParam = "hogehoge"
}

class "refBasePrim"
{
    custom string addClassParam = "refValue"
}




In [11]:
# サブレイヤーにVariant切り替えを入れる
v = subStage.GetPrimAtPath('/DefPrim').GetVariantSets().GetVariantSet('hogehoge')
v.SetVariantSelection('cube')

True

In [12]:
print(subStage.GetRootLayer().ExportToString())

#usda 1.0
(
    subLayers = [
        @usd/baseLayer.usda@
    ]
)

over "basePrim"
{
    custom int addAttr = 100
}

over "DefPrim" (
    variants = {
        string hogehoge = "cube"
    }
)
{
    over "sphere"
    {
        custom string addClassParam = "fugafuga"
    }

    over "childB"
    {
        custom string addClassParam = "setSubLayer"
    }
}




In [13]:
print(subStage.Flatten().ExportToString())

#usda 1.0
(
    doc = """Generated from Composed Stage of root layer 
"""
)

def "DefPrim"
{
    def Cube "cube"
    {
        custom int addAttr = 100
        custom string addClassParam = "hogehoge"
    }

    def "childA"
    {
        custom int addAttr = 100
        custom string addClassParam = "hogehoge"
    }

    def "childB"
    {
        custom int addAttr = 100
        custom string addClassParam = "setSubLayer"
    }

    def "childC"
    {
        custom int addAttr = 100
        custom string addClassParam = "hogehoge"
    }

    over "sphere"
    {
        custom string addClassParam = "fugafuga"
    }
}

class "basePrim"
{
    custom int addAttr = 100
    custom string addClassParam = "hogehoge"
}

class "refBasePrim"
{
    custom string addClassParam = "refValue"
}


